<a href="https://colab.research.google.com/github/marrownerd/mpi-openmp-hybrid-lab/blob/main/%D0%BB%D0%B0%D0%B1%D0%B02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# 1. Устанавливаем библиотеку OpenMPI
!apt-get update
!apt-get install -y openmpi-bin libopenmpi-dev

# 2. Проверяем, что все установилось (должно выдать версию)
!mpirun --version

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [31]:
%%writefile poisson.cpp
#include <mpi.h>
#include <omp.h>
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm>
#include <cstdio>

// === ПАРАМЕТРЫ ЗАДАЧИ ===
const double A_PARAM = 1e5;
const double EPS = 1e-8;
const double X0 = -1.0, Y0 = -1.0, Z0 = -1.0;
const double DX = 2.0, DY = 2.0, DZ = 2.0;

// Размер сетки (192 - хорошо для тестов в Colab)
const int Nx = 192;
const int Ny = 192;
const int Nz = 192;

// Точное решение и правая часть
double phi_exact(double x, double y, double z) { return x*x + y*y + z*z; }
double rho_func(double x, double y, double z) { return 6.0 - A_PARAM * phi_exact(x, y, z); }

int main(int argc, char** argv) {
    int rank, size, provided;

    // Инициализируем MPI с поддержкой многопоточности
    MPI_Init_thread(&argc, &argv, MPI_THREAD_FUNNELED, &provided);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Шаги по пространству
    double hx = DX / (Nx - 1), hy = DY / (Ny - 1), hz = DZ / (Nz - 1);
    double hx2 = hx*hx, hy2 = hy*hy, hz2 = hz*hz;
    double denom = 2.0/hx2 + 2.0/hy2 + 2.0/hz2 + A_PARAM;

    // Одномерная декомпозиция (разрезаем "тортик" на слои по оси X)
    int local_Nx = Nx / size;
    int remainder = Nx % size;
    int start_i = rank * local_Nx + std::min(rank, remainder);
    if (rank < remainder) local_Nx++;

    // Память: локальные слои + 2 теневых (ghost) слоя
    size_t layer_size = Ny * Nz;
    size_t total_size = (local_Nx + 2) * layer_size;

    std::vector<double> phi(total_size, 0.0);
    std::vector<double> phi_new(total_size, 0.0);
    std::vector<double> rho_arr(total_size, 0.0);

    // --- 1. Инициализация ---
    // i идет от 0 до local_Nx+1 (локальная индексация)
    #pragma omp parallel for collapse(2)
    for (int i = 0; i < local_Nx + 2; ++i) {
        for (int j = 0; j < Ny; ++j) {
            for (int k = 0; k < Nz; ++k) {
                // Перевод в глобальную координату
                int global_i = start_i + (i - 1);
                double x = X0 + global_i * hx;
                double y = Y0 + j * hy;
                double z = Z0 + k * hz;

                int idx = i * layer_size + j * Nz + k;
                rho_arr[idx] = rho_func(x, y, z);

                // Граничные условия (фиксируем значения на краях куба)
                if (global_i == 0 || global_i == Nx - 1 || j == 0 || j == Ny - 1 || k == 0 || k == Nz - 1) {
                    phi[idx] = phi_exact(x, y, z);
                    phi_new[idx] = phi[idx];
                }
            }
        }
    }

    double max_diff = 0.0;
    int it = 0;
    double start_time = MPI_Wtime();

    // --- 2. Основной цикл ---
    do {
        it++;
        max_diff = 0.0;
        MPI_Request reqs[4];
        int n_reqs = 0;

        // Асинхронный обмен границами (Halo exchange)
        // Отправляем свои крайние слои соседям
        if (rank > 0) {
            MPI_Isend(&phi[1 * layer_size], layer_size, MPI_DOUBLE, rank - 1, 0, MPI_COMM_WORLD, &reqs[n_reqs++]);
            MPI_Irecv(&phi[0 * layer_size], layer_size, MPI_DOUBLE, rank - 1, 1, MPI_COMM_WORLD, &reqs[n_reqs++]);
        }
        if (rank < size - 1) {
            MPI_Isend(&phi[local_Nx * layer_size], layer_size, MPI_DOUBLE, rank + 1, 1, MPI_COMM_WORLD, &reqs[n_reqs++]);
            MPI_Irecv(&phi[(local_Nx + 1) * layer_size], layer_size, MPI_DOUBLE, rank + 1, 0, MPI_COMM_WORLD, &reqs[n_reqs++]);
        }

        // Лямбда-функция для расчета одного слоя
        auto calc_layer = [&](int i) {
            int global_i = start_i + (i - 1);
            // Считаем только внутренние точки (границы фиксированы)
            if (global_i > 0 && global_i < Nx - 1) {
                double local_diff = 0.0;

                // Параллелим циклы по Y и Z с помощью OpenMP
                #pragma omp parallel for collapse(2) reduction(max: local_diff)
                for(int j=1; j<Ny-1; ++j) {
                    for(int k=1; k<Nz-1; ++k) {
                        int idx = i*layer_size + j*Nz + k;
                        // Формула Якоби (крест)
                        double val = ((phi[idx+layer_size] + phi[idx-layer_size])/hx2 +
                                      (phi[idx+Nz] + phi[idx-Nz])/hy2 +
                                      (phi[idx+1] + phi[idx-1])/hz2 - rho_arr[idx]) / denom;
                        phi_new[idx] = val;
                        local_diff = std::max(local_diff, std::abs(val - phi[idx]));
                    }
                }

                // Обновляем общий максимум (безопасно для потоков)
                #pragma omp critical
                {
                    if(local_diff > max_diff) max_diff = local_diff;
                }
            }
        };

        // (А) Считаем внутреннюю часть области (пока идут обмены)
        if (local_Nx > 2) {
            for (int i = 2; i < local_Nx; ++i) calc_layer(i);
        }

        // (Б) Ждем завершения обменов
        MPI_Waitall(n_reqs, reqs, MPI_STATUSES_IGNORE);

        // (В) Считаем границы (теперь у нас есть данные от соседей)
        if (local_Nx >= 1) calc_layer(1);
        if (local_Nx > 1) calc_layer(local_Nx);

        // Обновляем массив
        std::swap(phi, phi_new);

        // Собираем максимальную ошибку со всех процессов
        double global_max_diff;
        MPI_Allreduce(&max_diff, &global_max_diff, 1, MPI_DOUBLE, MPI_MAX, MPI_COMM_WORLD);
        max_diff = global_max_diff;

    } while (max_diff > EPS && it < 10000); // Ограничим 10000 итераций на всякий случай

    double end_time = MPI_Wtime();

    // --- 3. Проверка результата (считаем отклонение от точного решения) ---
    double local_err = 0.0;
    #pragma omp parallel for collapse(2) reduction(max: local_err)
    for(int i=1; i<=local_Nx; ++i) {
        for(int j=0; j<Ny; ++j) {
            for(int k=0; k<Nz; ++k) {
                int global_i = start_i + (i - 1);
                double x = X0 + global_i*hx, y = Y0 + j*hy, z = Z0 + k*hz;
                local_err = std::max(local_err, std::abs(phi[i*layer_size + j*Nz + k] - phi_exact(x,y,z)));
            }
        }
    }
    double global_err;
    MPI_Reduce(&local_err, &global_err, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        printf("RESULT: MPI=%d, OMP=%d, Time=%.4f sec, Error=%.2e, Iterations=%d\n",
               size, omp_get_max_threads(), end_time - start_time, global_err, it);
    }
    MPI_Finalize();
    return 0;
}

Overwriting poisson.cpp


In [32]:
# Компилируем с флагами оптимизации (-O3) и поддержкой OpenMP (-fopenmp)
!mpicxx poisson.cpp -o poisson_run -fopenmp -O3

# Проверяем, создался ли файл (должен быть в списке)
!ls -l poisson_run

-rwxr-xr-x 1 root root 104632 Dec 25 08:11 poisson_run


In [33]:
import subprocess
import os

# Функция запуска одного теста
def run_experiment(mpi_procs, omp_threads):
    print(f"⏳ Тест: {mpi_procs} MPI процесс(а) x {omp_threads} поток(а)...", end="", flush=True)

    env = os.environ.copy()
    env["OMP_NUM_THREADS"] = str(omp_threads)

    # Команда запуска MPI
    cmd = [
        "mpirun",
        "--allow-run-as-root",  # Разрешить запуск от root (нужно для Colab)
        "--oversubscribe",      # Разрешить запускать больше процессов, чем есть ядер
        "-np", str(mpi_procs),
        "./poisson_run"
    ]

    try:
        result = subprocess.run(cmd, env=env, capture_output=True, text=True, timeout=120)
        if result.returncode == 0:
            # Ищем строку результата в выводе
            for line in result.stdout.split('\n'):
                if "RESULT:" in line:
                    print(" ✅ Готово!")
                    print("   >> " + line)
                    return
            print(" ⚠️ Завершилось, но результат не найден.")
        else:
            print(" ❌ Ошибка!")
            print(result.stderr)
    except subprocess.TimeoutExpired:
        print(" ⏱️ Долго! (Таймаут)")

# === ЗАПУСК ЭКСПЕРИМЕНТОВ ===

print("\n--- ЭТАП 1: Масштабируемость (Только MPI) ---")
# В Colab всего 2 ядра, поэтому ускорение будет не идеальным, но цифры мы получим
for p in [1, 2, 4]:
    run_experiment(p, 1)

print("\n--- ЭТАП 2: Гибридный режим (Поиск лучшей конфигурации) ---")
# Эмулируем работу на "4 узлах" (всего 4 потока вычислений)
configs = [
    (1, 4), # 1 процесс, 4 потока
    (2, 2), # 2 процесса по 2 потока
    (4, 1)  # 4 процесса по 1 потоку
]

for mpi, omp in configs:
    run_experiment(mpi, omp)


--- ЭТАП 1: Масштабируемость (Только MPI) ---
⏳ Тест: 1 MPI процесс(а) x 1 поток(а)... ✅ Готово!
   >> RESULT: MPI=1, OMP=1, Time=0.7703 sec, Error=2.36e-09, Iterations=20
⏳ Тест: 2 MPI процесс(а) x 1 поток(а)... ✅ Готово!
   >> RESULT: MPI=2, OMP=1, Time=1.3199 sec, Error=2.36e-09, Iterations=20
⏳ Тест: 4 MPI процесс(а) x 1 поток(а)... ✅ Готово!
   >> RESULT: MPI=4, OMP=1, Time=1.0849 sec, Error=2.36e-09, Iterations=20

--- ЭТАП 2: Гибридный режим (Поиск лучшей конфигурации) ---
⏳ Тест: 1 MPI процесс(а) x 4 поток(а)... ✅ Готово!
   >> RESULT: MPI=1, OMP=4, Time=0.8720 sec, Error=2.36e-09, Iterations=20
⏳ Тест: 2 MPI процесс(а) x 2 поток(а)... ✅ Готово!
   >> RESULT: MPI=2, OMP=2, Time=2.3890 sec, Error=2.36e-09, Iterations=20
⏳ Тест: 4 MPI процесс(а) x 1 поток(а)... ✅ Готово!
   >> RESULT: MPI=4, OMP=1, Time=0.8624 sec, Error=2.36e-09, Iterations=20
